In [20]:
import numpy as np
from pathlib import Path
from intan_io import read_rhs
import intan_io2

basedir = Path("D:/data/sample_recordings")
# fp = basedir / "sample_2nd_lko_113126.rhs"
# fp = basedir / "sample_2nd_lko_240313_134130.rhs"
fp = basedir / "sample_2nd_lko_240320_113026.rhs"
# fp = basedir / "sample_2nd_lko_240320_113226.rhs"

# data = np.memmap(fp, mode="r")

# data = read_rhs(fp, ".rhs")

result, data_present = intan_io2.load_file(fp)


Reading Intan Technologies RHS Data File, Version 3.2

Found 4 amplifier channels.
Found 0 board ADC channels.
Found 0 board DAC channels.
Found 0 board digital input channels.
Found 0 board digital output channels.

File contains 60.002 seconds of data.  Amplifiers were sampled at 30.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Done!  Elapsed time: 1.4 seconds


In [24]:
intan_io2.print_all_channel_names(result)

B-025
B-027
B-029
B-031


In [21]:
from scipy import signal as sg


def highpass(signal, cutoff, fs=1250, order=6, ax=-1):
    nyq = 0.5 * fs

    b, a = sg.butter(order, cutoff / nyq, btype="highpass")
    yf = sg.filtfilt(b, a, signal, axis=ax)

    return yf

In [38]:
from scipy.stats import median_abs_deviation as mad
import matplotlib.pyplot as plt

# channel_names = ["B-025", "B-027", "B-029", "B-031"]
# channel_names = ["B-026", "B-028", "B-030", "B-031"]
channel_names = ["B-025", "B-027", "B-029", "B-031"]
# channel_names = ["B-025", "B-027", "B-029", "B-031"]

fig, axs = plt.subplots(len(channel_names), 1, sharex=True, sharey=True)
axs = axs.reshape(-1)

delta = 0
for i, channel_name in enumerate(channel_names):
    channel_found, signal_type, signal_index = intan_io2.find_channel_in_header(
        channel_name, result
    )
    y = result["amplifier_data"][signal_index, :]
    y = highpass(y, fs=30000, cutoff=500, order=3)
    mean_y = np.mean(y)
    std_y = np.std(y)
    mad_y = 1.4826 * mad(y)

    axs[i].plot(result["t"], y, color="gray", lw=0.6)
    axs[i].axhline(mean_y, color="k", ls="--")
    axs[i].axhline(mean_y - (2 * mad_y), color="g", ls="--")
    axs[i].axhline(mean_y - (3 * mad_y), color="r", ls="--")
    axs[i].axhline(mean_y - (5 * mad_y), color="magenta", ls="--")

axs[0].legend(
    ["Filtered Signal", "Mean", "2 MAD", "3 MAD", "5MAD"],
    loc="upper right",
    ncols=5,
)
axs[3].set_xlabel("Time (s)")
axs[3].set_ylabel("Voltage (uV)")
fig.suptitle("EywaNeuro Data")

Text(0.5, 0.98, 'EywaNeuro Data')

In [ ]:
channel_name = (
    "B-027"  # Change this variable and re-run cell to plot a different channel
)


if data_present:
    intan_io2.plot_channel(channel_name, result)

else:
    print("Plotting not possible; no data in this file")

### Rishika's data

In [25]:
from neuropy.io import NeuroscopeIO, BinarysignalIO
from pathlib import Path

basepath = Path(
    "D:/Data/ephys/Onix_2025-06-24_11-56-50/Onix_2025-06-24_11-56-50/Record Node 103/experiment1/recording1/continuous/OE_FPGA_Acquisition_Board-100.Rhythm Data"
)

io = NeuroscopeIO(xml_filename=basepath / "continuous.xml")

data = (
    BinarysignalIO(io.dat_filename, n_channels=67).get_signal(
        t_start=2758, t_stop=2760, channel_indx=[0, 2, 3, 4, 5, 6, 7]
    )
    # .rescale()
)

In [39]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(7, 1, sharex=True, sharey=True)
axs = axs.reshape(-1)

traces = data.traces

for i, ax in enumerate(axs):
    raw_trace = traces[i]
    y = highpass(raw_trace, fs=30000, cutoff=500, order=3)
    mean_y = np.mean(y)
    std_y = np.std(y)
    mad_y = 1.4826 * mad(y)

    axs[i].plot(data.time, y, color="gray", lw=0.6)
    axs[i].axhline(mean_y, color="k", ls="--")
    axs[i].axhline(mean_y - (2 * std_y), color="g", ls="--")
    axs[i].axhline(mean_y - (3 * std_y), color="r", ls="--")
    axs[i].axhline(mean_y - (5 * mad_y), color="magenta", ls="--")

    # ax.plot(traces[i])
    # ax.set_title(f"Channel {i}")
axs[0].legend(
    ["Filtered Signal", "Mean", "2 MAD", "3 MAD", "5MAD"],
    loc="upper right",
    ncols=5,
)

fig.suptitle("Rishika's Data")

Text(0.5, 0.98, "Rishika's Data")

In [41]:
4/5

0.8